In [2]:
import pandas as pd
import plotly.express as px

# 1. Carregar o dataframe detalhado
file_path = "audiencia_espelho_merged_segundos.csv"
df = None

try:
    df = pd.read_csv(file_path)
    print(f"Arquivo '{file_path}' carregado com sucesso!")
except FileNotFoundError:
    print(f"ERRO CRÍTICO: Arquivo '{file_path}' não encontrado.")
    print("Por favor, verifique o nome e o local do arquivo.")

# Prosseguir somente se o dataframe foi carregado com sucesso
if df is not None:
    # 2. Preparar e Filtrar os Dados
    required_cols = ['Retranca', 'Tipo', 'GLOBO']
    if not all(col in df.columns for col in required_cols):
        print(f"ERRO CRÍTICO: O arquivo precisa conter as colunas: {required_cols}")
    else:
        # Filtro pela coluna 'Tipo'
        tipos_a_remover = ['ABER', 'IMG', 'VINH']
        print(f"Removendo retrancas do tipo: {tipos_a_remover}")
        
        df_rank = df[
            df['Retranca'].notna() &
            (df['Retranca'] != "Não disponível") &
            (~df['Tipo'].isin(tipos_a_remover))
        ].copy()

        if df_rank.empty:
            print("Não há dados de 'Retranca' definidos para gerar o ranking após os filtros.")
        elif not pd.api.types.is_numeric_dtype(df_rank['GLOBO']):
            print("ERRO: Coluna 'GLOBO' não é numérica. Verifique os dados.")
        else:
            # --- ALTERAÇÃO: Bloco que renomeava duplicatas foi REMOVIDO ---
            # Agora o groupby() irá juntar as retrancas com mesmo nome.
            print("Matérias com nomes iguais serão agrupadas.")
            
            print("Calculando audiência média e duração por Matéria...")

            # 3. Agrupar e Calcular Métricas
            ranking_data = df_rank.groupby('Retranca').agg(
                audiencia_media_globo=('GLOBO', 'mean'),
                duracao_segundos=('GLOBO', 'count')
            ).reset_index()

            # O filtro de duração de 30s continua removido.
            df_plot = ranking_data.sort_values(by='audiencia_media_globo', ascending=True)
            
            num_retrancas = len(df_plot)
            print(f"Preparando gráfico de barras para {num_retrancas} matérias únicas.")
            
            chart_height = max(600, num_retrancas * 25)

            # 6. Criar o Gráfico de Barras Horizontais
            fig = px.bar(
                df_plot,
                x='audiencia_media_globo',
                y='Retranca',
                orientation='h',
                labels={'audiencia_media_globo': 'Audiência Média', 'Retranca': 'Matérias BDES'},
                hover_data=['duracao_segundos'],
                height=chart_height
            )
            
            fig.update_traces(
                texttemplate='%{x:.2f}', 
                textposition='inside',
                insidetextanchor='middle'
            )
            
            audiencia_media_fixa = 8.19
            fig.add_vline(
                x=audiencia_media_fixa,
                line_width=2,
                line_dash="dash",
                line_color="red",
                annotation_text=f"Média simples do dia: {audiencia_media_fixa:.1f}",
                annotation_position="bottom right",
                annotation_font_size=12,
                annotation_font_color="red"
            )

            # 7. Estilizar o Gráfico
            fig.update_layout(
                xaxis_title='Ranking Matérias BDES',
                title={
                    'text': "<b>Ranking Matérias BDES</b><br>" + \
                            "<span style='font-size: 14px;'>Análise de: 13/06/2025 | Fonte: Kantar - Ibope</span><br>" + \
                            "<span style='font-size: 12px;'><i>Desconsideradas retrancas do tipo: ABER, IMG, VINH</i></span>",
                    'y': 0.98,
                    'x': 0.5,
                    'xanchor': 'center',
                    'yanchor': 'top',
                    'font': {'size': 18}
                },
                margin=dict(l=350, t=130, b=50)
            )

            # 8. Salvar em HTML
            output_html_path = "ranking_de_materias_BDES.html"
            fig.write_html(output_html_path)
            print(f"\nGráfico de Ranking salvo como: '{output_html_path}'")
            print("Abra este arquivo HTML em um navegador para interagir com o gráfico.")
else:
    print("\nO DataFrame não foi carregado. A visualização não pode ser gerada.")

Arquivo 'audiencia_espelho_merged_segundos.csv' carregado com sucesso!
Removendo retrancas do tipo: ['ABER', 'IMG', 'VINH']
Matérias com nomes iguais serão agrupadas.
Calculando audiência média e duração por Matéria...
Preparando gráfico de barras para 52 matérias únicas.

Gráfico de Ranking salvo como: 'ranking_de_materias_BDES.html'
Abra este arquivo HTML em um navegador para interagir com o gráfico.
